# Crying baby with Importance and Rejection sampling

## Importing libraries

In [1]:
import numpy as np

## What state is the baby in?

##### X = (H, W, S, C)
H = 0 / 1, is it hungry?
<br>
W = 0 / 1, is it wet?
<br>
S = 0 / 1, does it smell?
<br>
C = 0 / 1, is it crying?

For example X = (1, 0, 1, 0) is a state of a hungry smelly baby

## Probabilities and distribution p(X)

##### Probability of a state X = (H, W, S, C)
p(X) = p(H, W, S, C) = p(H) * p(W) * p(S | W) * p(C | H,W)

Defining probabalities of individual realizations
- in reality, we might not have access to values
- what we need though, is access to calculate for each state X its probability p(X), even if only as a black box 

In [2]:
prob_H = 0.3
#Probability of a hungry baby (H=1)

prob_W = 0.4
#Probability of a wet baby (W=1)

prob_S = np.array([0.05, 0.7])
#Conditional probabilities of smelly baby (S=1)

prob_C = np.array([[0.2, 0.85], 
                   [0.8, 0.98]])
#Conditional probabilities of crying baby (C=1)

$$ prob\_S = \left( \begin{array}{cc} p(S=1 | W=0) & p(S=1 | W=1) \end{array}\right) $$

$$ prob\_C = \left( \begin{array}{cc} p(C=1 | H=0,W=0) & p(C=1 | H=0,W=1) \\ p(C=1 | H=1,W=0) & p(C=1 | H=1,W=1) \end{array}\right) $$

In [3]:
def probability_p(X):
    X = np.transpose(np.array(X))
    H_array = X[0]
    W_array = X[1]
    S_array = X[2]
    C_array = X[3]
    
    p_H = np.array([1-prob_H, prob_H])[H_array]
    p_W = np.array([1-prob_W, prob_W])[W_array]
    p_S = np.array([1-prob_S, prob_S])[S_array, W_array]
    p_C = np.array([1-prob_C, prob_C])[C_array, H_array, W_array]
    
    probability = p_H * p_W * p_S * p_C
    return probability
#Returns a probabilities p(X) for the given X = [[X_1], [X_2], [X_3], ... , [X_N]], where [X_i] = [H_i, W_i, S_i, C_i]

## Importance sampling

##### 1. Generate samples from an auxiliary distribution g

It should be easy to samples from g
<br>
=> let g be fair coin toss for each H, W, S, C => each state X_i has probability g(X_i) = 1/16

In [5]:
N = 100000    #Number of samples to be generated from g -> X_1, X_2, X_3, ... , X_N

IS_samples = np.random.randint(0, 2, (N, 4))
#Generated samples

print("Examples from the", N, "generated samples:")
print("X_1 =", IS_samples[0], "; X_2 =", IS_samples[1], "; X_3 =", IS_samples[2], "; X_4 =", IS_samples[3])

Examples from the 100000 generated samples:
X_1 = [0 0 0 1] ; X_2 = [1 0 0 1] ; X_3 = [1 1 0 1] ; X_4 = [0 0 0 0]


##### 2. Account for under/over representations by weights

$$ \omega_i = \frac{p(X_i)}{g(X_i)} $$

In [6]:
weights = probability_p(IS_samples) / (np.zeros(IS_samples.shape[0]) + 1/16)
#Array containing weight for each sample

##### 3. Estimate <.> of your interest

$$ <\phi> = \frac{\sum{\omega_i \phi(X_i)}}{\sum{\omega_i}} $$

Lets find out approximately how many nights that little devil is hungry / is wet / smells / spends crying

In [7]:
def IS_phi_function(X):
    #Define whatever function phi you want to
    output = X
    return output

IS_phi = IS_phi_function(IS_samples)

In [8]:
hungry = (np.sum(weights * IS_phi[:,0]) / np.sum(weights)) * 100
wet = (np.sum(weights * IS_phi[:,1]) / np.sum(weights)) * 100
smells = (np.sum(weights * IS_phi[:,2]) / np.sum(weights)) * 100
cries = (np.sum(weights * IS_phi[:,3]) / np.sum(weights)) * 100

print("The baby is hungry approximately", np.round(hungry,2), "% of the nights.")
print("The baby is wet approximately", np.round(wet,2), "% of the nights.")
print("The baby smells approximately", np.round(smells,2), "% of the nights.")
print("The baby cries approximately", np.round(cries,2), "% of the nights.")

The baby is hungry approximately 30.07 % of the nights.
The baby is wet approximately 39.99 % of the nights.
The baby smells approximately 31.07 % of the nights.
The baby cries approximately 58.62 % of the nights.


## Rejection sampling

##### 1. Generate samples X_1, X_2, X_3, ... , X_N

I am gonna do that the same way as for Importance sampling

In [14]:
N = 10000000    #Number of samples to be generated

RS_samples = np.random.randint(0, 2, (N, 4))
#Generated samples

print("Examples from the", N, "generated samples:")
print("X_1 =", IS_samples[0], "; X_2 =", IS_samples[1], "; X_3 =", IS_samples[2], "; X_4 =", IS_samples[3])

Examples from the 10000000 generated samples:
X_1 = [0 0 0 1] ; X_2 = [1 0 0 1] ; X_3 = [1 1 0 1] ; X_4 = [0 0 0 0]


##### 2. Generate u from uniform< 0 , Q(X) >

Q(X) > p(X) always and for for X => lets define Q(X) = 1.01
<br>
Why? Well p(X) can theoreticaly be <0,1> based on what prob_H, prob_W,... we set and I want this to work for all of them

In [15]:
u = np.random.uniform(0, 1.01, N)
#For each sample its own u is generated

##### 3. If - procedure

If p(X) > u, we accept X as an sample
<br>
If p(X) < u, we refuse X as an sample
<br>
<br>
Note: Discord notes "Sampling 14" claims it is the other way arround, but I don´t believe that

In [16]:
kept_samples = RS_samples[probability_p(RS_samples) > u]
print("Together we kept", kept_samples.shape[0], "out of", N, "samples. Thats", 100*kept_samples.shape[0]/N, "%")

Together we kept 618503 out of 10000000 samples. Thats 6.18503 %


##### 4. Calculate <.>

Once again, lets now calculate the approximate number of night for the baby to be hungry / wet / smelly / crying

In [17]:
def RS_phi_function(X):
    #Define whatever function phi you want to
    output = X
    return output

RS_phi = RS_phi_function(kept_samples)

In [18]:
print("The baby is hungry approximately", 100*np.round(np.mean(RS_phi[:,0]),2), "% of the nights.")
print("The baby is wet approximately", 100*np.round(np.mean(RS_phi[:,1]),2), "% of the nights.")
print("The baby smells approximately", 100*np.round(np.mean(RS_phi[:,2]),2), "% of the nights.")
print("The baby cries approximately", 100*np.round(np.mean(RS_phi[:,3]),2), "% of the nights.")

The baby is hungry approximately 30.0 % of the nights.
The baby is wet approximately 40.0 % of the nights.
The baby smells approximately 31.0 % of the nights.
The baby cries approximately 57.99999999999999 % of the nights.
